In [1]:
import numpy as np
import torch as pt
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn import metrics

from config import config_model, config_data
from data_handler import Dataset
from model import Model
from src.dataset import collate_batch_features, select_by_sid, select_by_max_ba, select_by_interface_types
from src.scoring import bc_scoring, bc_score_names

from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
rcParams['font.size'] = 14

In [ ]:
def setup_dataset(config_data, r_types_sel):
    # set up dataset
    dataset = Dataset("datasets/contacts_rr4A_64nn.h5")

    # selected structures
    sids_sel = np.genfromtxt("datasets/subunits_validation_highres_set.txt", dtype=np.dtype('U'))

    # filter dataset
    m = select_by_sid(dataset, sids_sel) # select by sids

    # data selection criteria
    m = select_by_sid(dataset, sids_sel) # select by sids
    m &= select_by_max_ba(dataset, config_data['max_ba'])  # select by max assembly count
    #m &= (dataset.sizes[:,0] <= config_data['max_size']) # select by max size
    m &= (dataset.sizes[:,1] >= config_data['min_num_res'])  # select by min size
    m &= select_by_interface_types(dataset, config_data['l_types'], np.concatenate(r_types_sel))  # select by interface type

    # update dataset selection
    dataset.update_mask(m)

    # set dataset types
    dataset.set_types(config_data['l_types'], config_data['r_types'])

    # debug print
    return dataset

In [ ]:
def eval_model(model, dataset, ids):
    p_l, y_l = [], []
    with pt.no_grad():
        for i in tqdm(ids):
            # get data
            X, ids_topk, q, M, y = dataset[i]

            # pack data and setup sink (IMPORTANT)
            X, ids_topk, q, M = collate_batch_features([[X, ids_topk, q, M]])

            # run model
            z = model(X.to(device), ids_topk.to(device), q.to(device), M.float().to(device))

            # prediction
            p = pt.sigmoid(z)

            # categorical predictions
            pc = pt.cat([1.0 - pt.max(p, dim=1)[0].unsqueeze(1), p], dim=1).cpu()
            yc = pt.cat([1.0 - pt.any(y > 0.5, dim=1).float().unsqueeze(1), y], dim=1).cpu()

            # data
            p_l.append(pc)
            y_l.append(yc)

    return p_l, y_l

In [ ]:
# select saved model
model_filepath = 'model_ckpt.pt'

# define device
device = pt.device("cuda")

# create model
model = Model(config_model)

# reload model
model.load_state_dict(pt.load(model_filepath, map_location=pt.device("cuda")))

# set model to inference
model = model.eval().to(device)

In [ ]:
# evaluate model
p_l, y_l = [], []
for i in range(len(config_data['r_types'])):
    # debug print
    print(config_data['r_types'][i])

    # load datasets
    dataset = setup_dataset(config_data, [config_data['r_types'][i]])
    print("dataset: {}".format(len(dataset)))

    # parameters
    N = min(len(dataset), 512)

    # run negative examples
    ids = np.arange(len(dataset))
    np.random.shuffle(ids)
    pi_l, yi_l = eval_model(model, dataset, ids[:N])

    # store evaluation results
    p_l.append(pi_l)
    y_l.append(yi_l)

In [ ]:
# parameters
class_names = ["carbohydrates", "cyclodextrins"]

# compute scores per class
scores = []
for i in range(len(y_l)):
    # extract class
    p = pt.cat(p_l[i], dim=0)[:,i+1]
    y = pt.cat(y_l[i], dim=0)[:,i+1]

    # compute scores
    s = bc_scoring(y.unsqueeze(1), p.unsqueeze(1)).squeeze().numpy()

    # compute F1 score
    f1 = metrics.f1_score(y.numpy().astype(int), p.numpy().round())

    # compute ratio of positives
    r = pt.mean(y)

    # store results
    scores.append(np.concatenate([s, [f1, r]]))

# pack data
scores = np.stack(scores).T

# make table
df = pd.DataFrame(data=np.round(scores,2), index=bc_score_names+['F1', 'r'], columns=class_names)

# save dataframe
df.to_csv("results/type_interface_search_scores.csv")

# display
#display(df)

In [ ]:
# plot
plt.figure(figsize=(5,4.5))
for i in range(len(y_l)):
    # get labels and predictions for class
    yi = pt.cat(y_l[i], dim=0)[:,i+1]
    pi = pt.cat(p_l[i], dim=0)[:,i+1]

    # compute roc and roc auc
    fpr, tpr, _ = metrics.roc_curve(yi.numpy(), pi.numpy())
    auc = metrics.auc(fpr, tpr)

    # update plot
    plt.plot(fpr, tpr, '-', label="{} (auc: {:.2f})".format(class_names[i], auc))

plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.0)
plt.legend(loc='best')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.tight_layout()
plt.savefig("results/type_interface_search_roc_auc.svg")
plt.savefig("results/type_interface_search_roc_auc.png", dpi=300)
plt.show()

In [ ]:
# plot
plt.figure(figsize=(5,4.5))
for i in range(len(y_l)):
    # get labels and predictions for class
    yi = pt.cat(y_l[i], dim=0)[:,i+1]
    pi = pt.cat(p_l[i], dim=0)[:,i+1]

    # compute roc and roc auc
    pre, rec, _ = metrics.precision_recall_curve(yi.numpy(), pi.numpy())
    auc = metrics.auc(rec, pre)

    # update plot
    plt.plot(rec, pre, '-', label="{} (auc: {:.2f})".format(class_names[i], auc))

plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.0)
plt.legend(loc='best')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.tight_layout()
plt.savefig("results/type_interface_search_pr_auc.svg")
plt.savefig("results/type_interface_search_pr_auc.png", dpi=300)
plt.show()

In [ ]:
# pack results
P = pt.cat([pt.cat(p, dim=0) for p in p_l], dim=0)
Y = pt.cat([pt.cat(y, dim=0) for y in y_l], dim=0)

# select only residues at interface
m = pt.any(Y[:,1:] > 0.5, dim=1)
Pi = P[m,1:]
Yi = Y[m,1:]

# pick same sampling of all classes
n = Yi.shape[1]
N = int(pt.min(pt.sum(Yi, dim=0)).item())
ids_unif = pt.from_numpy(np.concatenate([np.random.choice(pt.where(Yi[:,i] > 0.5)[0].numpy(), N, replace=False) for i in range(n)]))

# compute scores for each class
scores = []
for i in range(n):
    # extract class
    p = Pi[ids_unif,i]
    y = Yi[ids_unif,i]

    # compute scores
    s = bc_scoring(y.unsqueeze(1), p.unsqueeze(1)).squeeze().numpy()

    # compute F1 score
    f1 = metrics.f1_score(y.numpy().astype(int), p.numpy().round())

    scores.append(np.concatenate([s, [f1]]))

# pack data
scores = np.stack(scores).T

# make table
df = pd.DataFrame(data=np.round(scores,2), index=bc_score_names+['F1'], columns=class_names)

# save dataframe
df.to_csv("results/type_interface_identification_scores.csv")

# display
#display(df)

In [ ]:
# pack results
P = pt.cat([pt.cat(p, dim=0) for p in p_l], dim=0)
Y = pt.cat([pt.cat(y, dim=0) for y in y_l], dim=0)

# select only residues at interface
m = pt.any(Y[:,1:] > 0.5, dim=1)
Pi = P[m,1:]
Yi = Y[m,1:]

# pick same sampling of all classes
n = Yi.shape[1]
N = int(pt.min(pt.sum(Yi, dim=0)).item())
ids_l = [pt.from_numpy(np.random.choice(pt.where(Yi[:,i] > 0.5)[0].numpy(), N, replace=False)) for i in range(n)]

# compute scores for each class
C = pt.zeros((n,n))
for i in range(n):
    ids = pt.argmax(Pi[ids_l[i]], dim=1)
    for j,k in zip(ids, ids_l[i]):
        C[i,j] += Pi[k,j].round()


# normalize score
H = (C / pt.sum(C, dim=1).unsqueeze(1)).numpy()
#H = (C / pt.sum(C, dim=0).unsqueeze(0)).numpy()

# plot
plt.figure(figsize=(5, 5))
plt.imshow(H, origin='lower', cmap='BuGn', vmin=0.0, vmax=1.0)
plt.colorbar(fraction=0.046, pad=0.04)
plt.xticks(np.arange(n), class_names, rotation=90)
plt.yticks(np.arange(n), class_names)
for i in range(n):
    for j in range(n):
        v = H[i,j]
        if v > 0.1:
            plt.text(j,i,f"{v:.2f}", ha='center', va='center', color=[np.round(v-0.1)]*3)
plt.xlabel('predicted')
plt.ylabel('actual')
plt.tight_layout()
plt.savefig("results/type_interface_identification_most_confident_confusion_matrix.svg")
plt.savefig("results/type_interface_identification_most_confident_confusion_matrix.png", dpi=300)
plt.show()

In [ ]:
# pack results
P = pt.cat([pt.cat(p, dim=0) for p in p_l], dim=0)
Y = pt.cat([pt.cat(y, dim=0) for y in y_l], dim=0)

# select only residues at interface
m = pt.any(Y[:,1:] > 0.5, dim=1)
Pi = P[m,1:]
Yi = Y[m,1:]

# pick same sampling of all classes
n = Yi.shape[1]
N = int(pt.min(pt.sum(Yi, dim=0)).item())
ids_l = [pt.from_numpy(np.random.choice(pt.where(Yi[:,i] > 0.5)[0].numpy(), N, replace=False)) for i in range(n)]

# compute scores for each class
C = pt.zeros((n,n))
for i in range(n):
    ids = pt.argmax(Pi[ids_l[i]], dim=1)
    for j in range(n):
        for k in ids_l[i]:
            C[i,j] += Pi[k,j].round()


# normalize score
H = (C / pt.sum(C, dim=1).unsqueeze(1)).numpy()
#H = (C / pt.sum(C, dim=0).unsqueeze(0)).numpy()

# plot
plt.figure(figsize=(5, 5))
plt.imshow(H, origin='lower', cmap='BuGn', vmin=0.0, vmax=1.0)
plt.colorbar(fraction=0.046, pad=0.04)
plt.xticks(np.arange(n), class_names, rotation=90)
plt.yticks(np.arange(n), class_names)
for i in range(n):
    for j in range(n):
        v = H[i,j]
        if v > 0.1:
            plt.text(j,i,f"{v:.2f}", ha='center', va='center', color=[np.round(v-0.1)]*3)
plt.xlabel('predicted')
plt.ylabel('actual')
plt.tight_layout()
plt.savefig("results/type_interface_identification_most_confident_confusion_matrix.svg")
plt.savefig("results/type_interface_identification_most_confident_confusion_matrix.png", dpi=300)
plt.show()